Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# Eliminate repeat symbols.
symbols = set(etfs)

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

SDY XLRE IEUR XLF VPL PPLT SLY BIL JPST FXY FXB EBND UNG GDX SHY DIA VCSH UGA CANE BNDX IWD VCIT IWM MDY OIH UUP FXA QQQ JNK IYR DBB SHV XLC DBP EFA LQD FXF BWZ XLE SPY XLK AGG TLT XLV XOP XLY VDE EEM DBE BND XLI IWF VNQ NOBL XLU PFF USO IBND XLP GLD PGX VTI CYB DBA RSP CPER PALL SLV FXC XLB BWX VXUS CORN FXE 


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [9]:
# Sample symbol.
symbol = 'SPY'

In [10]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
# Sample
annual_returns[symbol]

28.10865222260681

In [11]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [12]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change(fill_method=None)
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.09239438029207436

In [13]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.05120591464078308

In [14]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   169.72
2015-02-28   179.26
2015-03-31   176.44
2015-04-30   178.18
2015-05-31   180.47
2015-06-30   176.80
2015-07-31   180.80
2015-08-31   169.78
2015-09-30   165.44
2015-10-31   179.52
2015-11-30   180.17
2015-12-31   177.06
2016-01-31   168.24
2016-02-29   168.11
2016-03-31   179.41
2016-04-30   180.12
2016-05-31   183.18
2016-06-30   183.82
2016-07-31   190.52
2016-08-31   190.75
2016-09-30   190.76
2016-10-31   187.46
2016-11-30   194.36
2016-12-31   198.30
2017-01-31   201.85
2017-02-28   209.78
2017-03-31   210.04
2017-04-30   212.13
2017-05-31   215.12
2017-06-30   216.49
2017-07-31   220.94
2017-08-31   221.59
2017-09-30   226.05
2017-10-31   231.38
2017-11-30   238.45
2017-12-31   241.34
2018-01-31   254.95
2018-02-28   245.68
2018-03-31   238.94
2018-04-30   240.18
2018-05-31   246.02
2018-06-30   247.43
2018-07-31   256.60
2018-08-31   264.79
2018-09-30   266.36
2018-10-31   247.96
2018-11-30   252.55
2018-12-31   230.32
2019-01-31   248.76
2019-02-28   25

In [15]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change(fill_method=None)
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2023-08-31   -0.02
2023-09-30   -0.05
2023-10-31   -0.02
2023-11-30    0.09
2023-12-31    0.05
Freq: M, Name: SPY, Length: 108, dtype: float64

In [16]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.17342412823142983

In [17]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [18]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [19]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')